In [1]:
import pandas as pd
import numpy as np
import os
from tqdm.notebook import tqdm
from scipy.stats import skew, kurtosis

In [2]:
import re

_nsre = re.compile('([0-9]+)')
def natural_sort_key(s):
    return [int(text) if text.isdigit() else text.lower()
            for text in re.split(_nsre, s)]   

def sort_files(files):
    files_sorted = []
    for file in files:
        if 'einit' not in file:
             files_sorted.append(file)
    return files_sorted

In [3]:
data_type = "train/"#input("test/ or train_data/: ", )

if data_type=='test/':
    energies = [0.1, 0.3, 0.6] + list(range(1, 10))
    title = "Energies processing..."  
elif data_type=='train/':
    energy = ''
    energies = [0]
    title = "Train data processing..."
size = len(energies)

In [4]:
geometry = pd.read_csv('geometry/sipm_pos_xyz.csv')

In [ ]:
thr_array = np.linspace(0, 95, 20)
thr_array[0] = 2

eps = 0
for en_value in tqdm(energies, title):
    
    if data_type=='test_data/':
        energy = '/{}_{}/'.format(type_of_particle, en_value)  
        
    path = "/mnt/cephfs/ml_data/TAO_detsim_J22/"

    files_data = os.listdir(path+data_type+energy)
    files_data.sort(key=natural_sort_key)

    df = pd.DataFrame()
    for k in tqdm(range(500), "Files processing...", leave=False):
        data = np.load(path+data_type+energy+files_data[k], allow_pickle=True)
        hits = data['hits']
        
        charges_arrays = []
        fhts_arrays = []
        pmids_arrays = []

        for j in tqdm(range(hits.shape[1]), leave=False):

            hitt = {}
            charges = []
            fhts = []
            pmids = []

            for i in range(hits[0, j].shape[0]):
                ind = hits[0, j][i]
                try:
                    hitt[ind].append(hits[1, j][i])
                except KeyError:
                    hitt[ind] = []
                    hitt[ind].append(hits[1, j][i])

            for key in hitt:
                charges.append(len(hitt[key]))
                fhts.append(np.min(hitt[key]))
                pmids.append(key)

            charges_arrays.append(np.array(charges))
            fhts_arrays.append(np.array(fhts))
            pmids_arrays.append(np.array(pmids))

        charges_arrays = np.array(charges_arrays, dtype=object)
        fhts_arrays = np.array(fhts_arrays, dtype=object)
        pmids_arrays = np.array(pmids_arrays, dtype=object)
        
        NEvents = charges_arrays.shape[0]
        
        ht_mean = [fhts_arrays[i].mean() for i in range(NEvents)]
        ht_std = [fhts_arrays[i].std() for i in range(NEvents)]
        ht_skew = [skew(fhts_arrays[i]) for i in range(NEvents)]
        ht_kurtosis = [kurtosis(fhts_arrays[i]) for i in range(NEvents)]

        pe_mean = [charges_arrays[i].mean() for i in range(NEvents)]
        pe_std = [charges_arrays[i].std() for i in range(NEvents)]
        pe_skew = [skew(charges_arrays[i]) for i in range(NEvents)]
        pe_kurtosis = [kurtosis(charges_arrays[i]) for i in range(NEvents)]

        x_cc = np.zeros(NEvents)
        y_cc = np.zeros(NEvents)
        z_cc = np.zeros(NEvents)
        
        x_cht = np.zeros(NEvents)
        y_cht = np.zeros(NEvents)
        z_cht = np.zeros(NEvents)
        
        accum_charge = np.zeros(NEvents)
        nPMTs = np.zeros(NEvents)

        for i in tqdm(range(NEvents), "CC and CHT computing", leave=False):
            sipm_x = np.array(geometry['X'][pmids_arrays[i]]) / 1000.
            sipm_y = np.array(geometry['Y'][pmids_arrays[i]]) / 1000.
            sipm_z = np.array(geometry['Z'][pmids_arrays[i]]) / 1000.

            x_cc[i] = np.sum(sipm_x * charges_arrays[i]) / np.sum(charges_arrays[i])
            y_cc[i] = np.sum(sipm_y * charges_arrays[i]) / np.sum(charges_arrays[i])
            z_cc[i] = np.sum(sipm_z * charges_arrays[i]) / np.sum(charges_arrays[i])
            
            x_cht[i] = np.sum(sipm_x / (fhts_arrays[i] + eps)) / np.sum(1 / (fhts_arrays[i] + eps))
            y_cht[i] = np.sum(sipm_y / (fhts_arrays[i] + eps)) / np.sum(1 / (fhts_arrays[i] + eps))
            z_cht[i] = np.sum(sipm_z / (fhts_arrays[i] + eps)) / np.sum(1 / (fhts_arrays[i] + eps))
            
            accum_charge[i] = np.sum(charges_arrays[i])
            nPMTs[i] = int(charges_arrays[i].shape[0])
        
        R_cc = (x_cc**2 + y_cc**2 + z_cc**2)**0.5
        rho_cc = (x_cc**2 + y_cc**2)**0.5
        theta_cc = np.arctan2((x_cc**2 + y_cc**2)**0.5, z_cc)
        phi_cc = np.arctan2(y_cc, x_cc)
        gamma_z_cc = z_cc / (x_cc**2 + y_cc**2)**0.5
        gamma_y_cc = y_cc / (z_cc**2 + x_cc**2)**0.5
        gamma_x_cc = x_cc / (z_cc**2 + y_cc**2)**0.5
        jacob_cc = R_cc**2 * np.sin(theta_cc)

        R_cht = (x_cht**2 + y_cht**2 + z_cht**2)**0.5
        rho_cht = (x_cht**2 + y_cht**2)**0.5
        theta_cht = np.arctan2((x_cht**2 + y_cht**2)**0.5, z_cht)
        phi_cht = np.arctan2(y_cht, x_cht)
        gamma_z_cht = z_cht / (x_cht**2 + y_cht**2)**0.5
        gamma_y_cht = y_cht / (z_cht**2 + x_cht**2)**0.5
        gamma_x_cht = x_cht / (z_cht**2 + y_cht**2)**0.5
        jacob_cht = R_cht**2 * np.sin(theta_cht)

        ht_ps = []
        pe_ps = []
        for thr in tqdm(thr_array, "FHT and PE percentiles", leave=False):
            ht_ps.append([np.percentile(fhts_arrays[i], thr) for i in range(NEvents)])
            pe_ps.append([np.percentile(charges_arrays[i], thr) for i in range(NEvents)])
        ht_ps = np.array(ht_ps)
        
        diff_ht_ps = []
        for i in range(1, len(ht_ps)):
                diff_ht_ps.append(ht_ps[i] - ht_ps[i-1])
        
        features_df = pd.DataFrame()
        features_df['AccumCharge'] = accum_charge
        features_df['nPMTs'] = nPMTs

        features_df['R_cc'] = R_cc
        features_df['rho_cc'] = rho_cc
        features_df['x_cc'] = x_cc
        features_df['y_cc'] = y_cc
        features_df['z_cc'] = z_cc
        features_df['gamma_z_cc'] = gamma_z_cc
        features_df['gamma_y_cc'] = gamma_y_cc
        features_df['gamma_x_cc'] = gamma_x_cc
        features_df['theta_cc'] = theta_cc
        features_df['phi_cc'] = phi_cc
        features_df['jacob_cc'] = jacob_cc

        features_df['R_cht'] = R_cht
        features_df['rho_cht'] = rho_cht
        features_df['x_cht'] = x_cht
        features_df['y_cht'] = y_cht
        features_df['z_cht'] = z_cht
        features_df['gamma_z_cht'] = gamma_z_cht
        features_df['gamma_y_cht'] = gamma_y_cht
        features_df['gamma_x_cht'] = gamma_x_cht
        features_df['theta_cht'] = theta_cht
        features_df['phi_cht'] = phi_cht
        features_df['jacob_cht'] = jacob_cht

        features_df['ht_std'] = ht_std
        features_df['ht_mean'] = ht_mean
        features_df['ht_skew'] = ht_skew
        features_df['ht_kurtosis'] = ht_kurtosis

        features_df['pe_std'] = pe_std
        features_df['pe_mean'] = pe_mean
        features_df['pe_skew'] = pe_skew
        features_df['pe_kurtosis'] = pe_kurtosis

        features_df['ht_2p'] = ht_ps[0]
        features_df['ht_5p'] = ht_ps[1]
        features_df['ht_10p'] = ht_ps[2]
        features_df['ht_15p'] = ht_ps[3]
        features_df['ht_20p'] = ht_ps[4]
        features_df['ht_25p'] = ht_ps[5]
        features_df['ht_30p'] = ht_ps[6]
        features_df['ht_35p'] = ht_ps[7]
        features_df['ht_40p'] = ht_ps[8]
        features_df['ht_45p'] = ht_ps[9]
        features_df['ht_50p'] = ht_ps[10]
        features_df['ht_55p'] = ht_ps[11]
        features_df['ht_60p'] = ht_ps[12]
        features_df['ht_65p'] = ht_ps[13]
        features_df['ht_70p'] = ht_ps[14]
        features_df['ht_75p'] = ht_ps[15]
        features_df['ht_80p'] = ht_ps[16]
        features_df['ht_85p'] = ht_ps[17]
        features_df['ht_90p'] = ht_ps[18]
        features_df['ht_95p'] = ht_ps[19]

        features_df['ht_5-2p'] = diff_ht_ps[0]
        features_df['ht_10-5p'] = diff_ht_ps[1]
        features_df['ht_15-10p'] = diff_ht_ps[2]
        features_df['ht_20-15p'] = diff_ht_ps[3]
        features_df['ht_25-20p'] = diff_ht_ps[4]
        features_df['ht_30-25p'] = diff_ht_ps[5]
        features_df['ht_35-30p'] = diff_ht_ps[6]
        features_df['ht_40-35p'] = diff_ht_ps[7]
        features_df['ht_45-40p'] = diff_ht_ps[8]
        features_df['ht_50-45p'] = diff_ht_ps[9]
        features_df['ht_55-50p'] = diff_ht_ps[10]
        features_df['ht_60-55p'] = diff_ht_ps[11]
        features_df['ht_65-60p'] = diff_ht_ps[12]
        features_df['ht_70-65p'] = diff_ht_ps[13]
        features_df['ht_75-70p'] = diff_ht_ps[14]
        features_df['ht_80-75p'] = diff_ht_ps[15]
        features_df['ht_85-80p'] = diff_ht_ps[16]
        features_df['ht_90-85p'] = diff_ht_ps[17]
        features_df['ht_95-90p'] = diff_ht_ps[18]
        
        features_df['pe_2p'] = pe_ps[0]
        features_df['pe_5p'] = pe_ps[1]
        features_df['pe_10p'] = pe_ps[2]
        features_df['pe_15p'] = pe_ps[3]
        features_df['pe_20p'] = pe_ps[4]
        features_df['pe_25p'] = pe_ps[5]
        features_df['pe_30p'] = pe_ps[6]
        features_df['pe_35p'] = pe_ps[7]
        features_df['pe_40p'] = pe_ps[8]
        features_df['pe_45p'] = pe_ps[9]
        features_df['pe_50p'] = pe_ps[10]
        features_df['pe_55p'] = pe_ps[11]
        features_df['pe_60p'] = pe_ps[12]
        features_df['pe_65p'] = pe_ps[13]
        features_df['pe_70p'] = pe_ps[14]
        features_df['pe_75p'] = pe_ps[15]
        features_df['pe_80p'] = pe_ps[16]
        features_df['pe_85p'] = pe_ps[17]
        features_df['pe_90p'] = pe_ps[18]
        features_df['pe_95p'] = pe_ps[19]
        
        primaries = data['primaries']
        primaries = pd.DataFrame(
            primaries, columns=['edep', 'edepX', 'edepY', 'edepZ'])
        primaries['edepR'] = np.sqrt(
            primaries['edepX']**2 + primaries['edepY']**2 + primaries['edepZ']**2
        )
        features_df = pd.concat([features_df, primaries], axis=1)
        df = df.append(features_df)
        
    if data_type=='test/':
        print("Done.")
        df.to_csv(
            f'{path}processed_data/ProcessedTest/{en_value}MeV.csv.gz',
            index=False, compression='gzip')
    elif data_type=='train/':
        print("Done.")
        df.to_csv(
            f'{path}processed_data/ProcessedTrain/ProcessedTrain6.csv.gz',
            index=False, compression='gzip')

Train data processing...:   0%|          | 0/1 [00:00<?, ?it/s]

Files processing...:   0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

CC and CHT computing:   0%|          | 0/1000 [00:00<?, ?it/s]

FHT and PE percentiles:   0%|          | 0/20 [00:00<?, ?it/s]

...

FHT and PE percentiles:   0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

CC and CHT computing:   0%|          | 0/1000 [00:00<?, ?it/s]

FHT and PE percentiles:   0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]